In [1]:
from typing import List, Union
from time import sleep
from abc import ABC, abstractmethod

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.common.by import By
from item import Item
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import pandas as pd
from bs4 import BeautifulSoup

/Library/Python/3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
class Parser(ABC):
    @abstractmethod
    def get_product_list(self) -> List[Item]:
        pass

    def get_page_content(self,
                         url: str,
                         scroll: bool = False,
                         scroll_timeout: Union[int, float] = 0.3) -> str:
        options = Options()
        options.add_argument("window-size=1920,1080")
        service = Service(ChromeDriverManager().install())

        all_content = []

        with (webdriver.Chrome(service=service, options=options) as driver):
            driver.get(url)
            if scroll:
                current_page = 1
                max_pages = 52
                while current_page <= max_pages:
                    sleep(2)
                    for i in range(18):
                        sleep(scroll_timeout)
                        driver.execute_script(f"window.scrollTo({i * 300}, {(i + 1) * 300})")
                        sleep(0.4)

                    page_content = driver.page_source
                    all_content.append(page_content)
                    sleep(1)
                    if current_page < 5:
                        try:
                            next_button = WebDriverWait(driver, 20).until(
                                EC.element_to_be_clickable((By.XPATH, '//*[@id="main"]/div/div[2]/div/article/div/section[3]/div[2]/a[9]'))
                            )
                            next_button.click()
                        except Exception as e:
                            print(f"Не удалось найти кнопку 'Следующая' или произошла ошибка - 9: {e}") 
                    elif current_page >= 5:
                        try:
                            next_button = WebDriverWait(driver, 20).until(
                                EC.element_to_be_clickable((By.XPATH, '//*[@id="main"]/div/div[2]/div/article/div/section[3]/div[2]/a[11]'))
                            )
                            next_button.click()
                        except Exception as e:
                            print(f"Не удалось найти кнопку 'Следующая' или произошла ошибка - 11: {e}")
                    else:
                            break
                    current_page += 1
                    driver.execute_script("window.scrollTo(0, 0);")
                    
        return all_content

In [3]:
class DogsParser(Parser):
    def __init__(self) -> None:
        self.url = "https://www.petshop.ru/adverts/dogs/"
        self.source = "petshop.ru"
        self.scroll = True

    def clean_text(self, text: str) -> str:
        text = text.replace(';', '').strip()
        text = re.sub(r'\s+', ' ', text)
        return text

    def get_product_list(self) -> None:
        content_list = super().get_page_content(url=self.url, scroll=self.scroll)

        names = []
        descriptions = []
        prices = []
        breeds = []
        regions = []
        dates = []
        images = []

        for content in content_list:
            soup = BeautifulSoup(content, features="lxml")

            raw_list = soup.find_all('div', attrs={'class': 'articles-item ng-scope'})

            for idx, raw in enumerate(raw_list, 1):
                soup = BeautifulSoup(str(raw), features='lxml')
                name = self.clean_text(soup.find('a', attrs={'class': "ng-binding"}).get_text())
                description = self.clean_text(soup.find('div', attrs={'class': "text ng-binding"}).get_text())
                price = self.clean_text(soup.find('div', attrs={'class': 'price ng-scope'}).find('span', attrs={
                    'class': 'ng-binding'}).get_text()) if soup.find('div',
                                                                     attrs={'class': 'price ng-scope'}) else "N/A"
                breed = self.clean_text(soup.find('div', attrs={'class': 'breed'}).find('span', attrs={
                    'class': 'ng-binding'}).get_text()) if soup.find('div', attrs={'class': 'breed'}) else "N/A"
                region = self.clean_text(soup.find('div', attrs={'class': 'region'}).find('span', attrs={
                    'class': 'ng-binding'}).get_text()) if soup.find('div', attrs={'class': 'region'}) else "N/A"
                date = self.clean_text(soup.find('div', attrs={'class': 'date'}).find('span', attrs={
                    'class': 'ng-binding'}).get_text()) if soup.find('div', attrs={'class': 'date'}) else "N/A"
                img_div = soup.find('div', attrs={'class': 'articles-img'})
                if img_div:
                    img_tag = img_div.find('img')
                    img = img_tag['ng-src'] if img_tag and 'ng-src' in img_tag.attrs else "N/A"
                else:
                    img = "N/A"

                names.append(name)
                descriptions.append(description)
                prices.append(price)
                breeds.append(breed)
                regions.append(region)
                dates.append(date)
                images.append(img)

        data = {
            'name': names,
            'description': descriptions,
            'price': prices,
            'breed': breeds,
            'region': regions,
            'date': dates,
            'image_url': images
        }
        df = pd.DataFrame(data)

        df.to_csv('dogs_dataset.csv', index=False, encoding='utf-8-sig')

        print('Data saved to dogs_dataset.csv in structured format.')


In [4]:
parser = DogsParser()
parser.get_product_list()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.265)
Stacktrace:
0   chromedriver                        0x00000001008ba138 cxxbridge1$str$ptr + 3653888
1   chromedriver                        0x00000001008b2988 cxxbridge1$str$ptr + 3623248
2   chromedriver                        0x0000000100318968 cxxbridge1$string$len + 89228
3   chromedriver                        0x00000001002f3e44 core::str::slice_error_fail::ha0e52dbcb60e6bae + 3780
4   chromedriver                        0x0000000100382c84 cxxbridge1$string$len + 524200
5   chromedriver                        0x0000000100395b60 cxxbridge1$string$len + 601732
6   chromedriver                        0x0000000100351564 cxxbridge1$string$len + 321672
7   chromedriver                        0x00000001003521b4 cxxbridge1$string$len + 324824
8   chromedriver                        0x0000000100884fc0 cxxbridge1$str$ptr + 3436424
9   ch

In [42]:
df = pd.read_csv('dogs_dataset.csv')

In [43]:
df

,name,description,price,breed,region,date,image_url
0,Малый брабансон красивый мальчик,МОСКВА Рыжий МАЛЬЧИК пти-брабансон/малый браба...,50000 - 50000 р.,Брюссельский гриффон,Москва,08.10.2024,//6kcmxu3d7l.a.trbcdn.net/upload/files-new/17/...
1,Щенки ротвейлера,"!комментарии не отслеживаю,звоните.Подрощенные...",45000 - 60000 р.,Ротвейлер,Екатеринбург,05.10.2024,//6kcmxu3d7l.a.trbcdn.net/upload/files-new/f4/...
2,Австралийский шелковистый терьер,Всем добра ! Возможно вы долго искали Силки! О...,60000 р.,Австралийский келпи,Санкт-Петербург,04.10.2024,//6kcmxu3d7l.a.trbcdn.net/upload/files-new/d0/...
3,Австралийский Шелковистый терьер,"Если вы выбираете себе питомца, то настало вре...",60000 р.,Австралийская овчарка,Москва,03.10.2024,//6kcmxu3d7l.a.trbcdn.net/upload/files-new/00/...
4,Маленький Малёк ищет новую семью,Ищем новую семью нашей Молли! Расскажу историю...,NaN,Не указано,Москва,01.10.2024,//6kcmxu3d7l.a.trbcdn.net/upload/files-new/42/...
...,...,...,...,...,...,...,...
1035,Мопсики,Малыши готовы к РЕЗЕРВУ. К переезду готовы пос...,35000 - 50000 р.,Мопс,Санкт-Петербург,05.09.2022,//6kcmxu3d7l.a.trbcdn.net/upload/files-new/4e/...
1036,Продаются щенки Джек Рассел Терьера,Продаются одна девочка и два мальчика Джек Рас...,25000 р.,Джек рассел терьер,Краснодар,04.09.2022,//6kcmxu3d7l.a.trbcdn.net/upload/files-new/d7/...
1037,Продаются щенки Джек Рассел Терьера,Продаются одна девочка и два мальчика Джек Рас...,25000 р.,Джек рассел терьер,Краснодар,04.09.2022,//6kcmxu3d7l.a.trbcdn.net/upload/files-new/a1/...
1038,Щенки Сибаину от чемпиона мира,В продаже щенки Сибаину с безупречной родослов...,70000 р.,Сиба-ину,Москва,03.09.2022,//6kcmxu3d7l.a.trbcdn.net/upload/files-new/b0/...
